In [1]:
import ee
import geemap
import os


In [2]:
# geemap.update_package()

In [3]:
Map = geemap.Map()


In [4]:
# mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
# clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
# landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
# landsat_vis = {
#     'min': 0,
#     'max': 1,
#     'bands': ['b5', 'b4', 'b3']
# }
# Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
# Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
# Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
# Map

In [5]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
Map.addLayer(countries, {}, 'countries')


In [6]:
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

In [7]:
region = ee.FeatureCollection('users/franciscoalbornoz/Aysen')
Map.addLayer(region, {}, 'Aysen')

In [8]:
start_date = '2018-12-01'
end_date = '2019-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)

In [9]:
median = l8.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')
Map.centerObject(roi, 10)
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [11]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [12]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [13]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [14]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

In [15]:
greenest = withNDVI.qualityMosaic('NDVI').clip(region)

In [16]:
greenest.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

In [17]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [18]:
Map.addLayer(greenest, visParams, 'Greenest pixel')
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [19]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [20]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]
# paleta de 18 colores


In [21]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [22]:
landcover_2018.bandNames().getInfo()

['b1']

In [23]:
clasificacion_2018.bandTypes().getInfo()


{'b1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [24]:
# land2018 = landcover_2018.set('classification_class_values', class_values)
# land2018f = land2018.set('classification_class_palette', class_palette)

In [25]:
# class2018 = clasificacion_2018.set('classification_class_values', class_values)
# class2018f = class2018.set('classification_class_palette', class_palette)

In [26]:
# Map.addLayer(land2018f, {}, 'landcover_prov_coyhaique' )
# Map.addLayer(class2018f,{},'clasificacion 2018')

In [27]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [28]:
work_dir = os.path.expanduser('../Data/shapes/')

val_shp= os.path.join(work_dir, 'validacion.shp')
merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [29]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [30]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')


In [31]:
merged_shp

'../Data/shapes/roi_merged.shp'

In [32]:
dataset = greenest.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [33]:
datasetval = greenest.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [34]:
dataset.size().getInfo()

20322

In [35]:
datasetval.size().getInfo()

4679

In [36]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.10622850805521011, 'B10': 277.4976806640625, 'B11': 277.0090026855469, 'B2': 0.08158686012029648, 'B3': 0.05989211052656174, 'B4': 0.03704666346311569, 'B5': 0.3357214033603668, 'B6': 0.08223237097263336, 'B7': 0.02972152642905712, 'B8': 0.04830099269747734, 'B9': 0.0024417066015303135, 'BQA': 2720, 'ID': 1, 'NDVI': 0.801234781742096, 'date': 20190210, 'doy': 41, 'month': 2}}


In [37]:
# Guardamos las bandas
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

# Esta propiedad guarda las clases de landcover
label = 'ID'

# Overlay the points on the imagery to get training.
# sample = mosaico.select(bands).sampleRegions(**{
#   'collection': dataset,
#   'properties': ['RF'],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [38]:
# print(training.first().getInfo())

In [39]:
# print(validation.first().getInfo())

In [40]:
randomforest = ee.Classifier.smileRandomForest(100).train(dataset, label, bands)
resultRF = greenest.select(bands).classify(randomforest)

In [41]:
SVM = ee.Classifier.libsvm('Voting','C_SVC','RBF', gamma = 0.5 , cost = 10 )
trained = SVM.train(dataset, label, bands)
classifiedSVM = greenest.classify(trained)

In [42]:
smc = ee.Classifier.smileCart().train(dataset, label, bands)
classifiedsmc = greenest.select(bands).classify(smc)

In [43]:
# Clasificando con las mismas bandas de entrenamiento
# result1 = greenest.select(bands).classify(randomforest)
# result2 = greenest.select(bands).classify(SVM)
# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'classificado')
# Map

In [44]:
# result.getInfo()

In [45]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [46]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [47]:
landcover_randomforest = resultRF.set('classification_class_values', class_values)
landcover_randomforest = landcover_randomforest.set('classification_class_palette', class_palette)

In [48]:
Map.addLayer(landcover_randomforest, {}, 'Land cover Random Forest Región')

In [49]:
landcover_svm = classifiedSVM.set('classification_class_values', class_values)
landcover_svm = landcover_svm.set('classification_class_palette', class_palette)

In [50]:
Map.addLayer(landcover_svm, {}, 'Land cover SVM Región')

In [51]:
landcover_cart = classifiedsmc.set('classification_class_values', class_values)
landcover_cart = landcover_cart.set('classification_class_palette', class_palette)

In [52]:
Map.addLayer(landcover_cart, {}, 'Land cover SCART Región')

In [53]:
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

RF

In [54]:
train_accuracy = randomforest.confusionMatrix()

In [55]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 890, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1118, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 4319, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 2302, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 139, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1331, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 881, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 72, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 320, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 232, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4473, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 366, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1107, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 306, 0, 0, 0],
 [0, 

In [56]:
train_accuracy.accuracy().getInfo()

0.9997047534691468

In [57]:
train_accuracy.kappa().getInfo()

0.9996617165612868

In [58]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.9991063449508489],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0.972972972972973],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0.9990974729241877],
 [0.9935064935064936],
 [1],
 [1],
 [1]]

In [59]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9988776655443322,
  1,
  0.9993058769088385,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0.9953917050691244,
  0.9991423670668954]]

In [60]:
validacion = datasetval.classify(randomforest)

In [61]:
validacion.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 0.1297835111618042,
  'B10': 278.9761047363281,
  'B11': 276.3595275878906,
  'B2': 0.10309768468141556,
  'B3': 0.06259501725435257,
  'B4': 0.046153344213962555,
  'B5': 0.0707976296544075,
  'B6': 0.03525298461318016,
  'B7': 0.027852408587932587,
  'B8': 0.05712661147117615,
  'B9': 0.0011301300255581737,
  'BQA': 2976,
  'ID': 3,
  'NDVI': 0.21072322130203247,
  'classification': 3,
  'date': 20190321,
  'doy': 80,
  'month': 3}}

In [62]:
testAccuracy = validacion.errorMatrix('ID', 'classification')

In [63]:
testAccuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 14],
 [0, 0, 0, 41, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 16, 0, 0, 0],
 [0, 0, 0, 371, 468, 0, 0, 3, 9, 0, 0, 0, 0, 10, 0, 24, 0, 0, 0],
 [0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 39, 4, 108, 0, 0, 240, 0, 0, 32, 0, 0, 80, 0, 0, 0, 0, 82],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 52, 0, 2, 1, 22, 19, 0, 0, 23, 9, 28, 22, 4, 0, 0, 0, 24],
 [0, 5, 48, 0, 0, 9, 132, 0, 0, 0, 0, 64, 7, 23, 56, 0, 32, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 10, 0, 0, 0, 0],
 [0, 6, 151, 0, 4, 10, 11, 4, 0, 0, 5, 4, 3, 54, 0, 0, 1, 0, 170],
 [0, 588, 9, 0, 10, 1, 255, 0, 0, 0, 0, 10, 37, 0, 486, 0, 132, 30, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [64]:
testAccuracy.accuracy().getInfo()

0.36524898482581747

In [65]:
testAccuracy.kappa().getInfo()

0.30119699286957563

In [66]:
testAccuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.7540983606557377],
 [0.6212121212121212],
 [0.5288135593220339],
 [0.6666666666666666],
 [1],
 [0.41025641025641024],
 [0.08333333333333333],
 [1],
 [0.1111111111111111],
 [0.16976127320954906],
 [0.8],
 [0.1276595744680851],
 [0.3119383825417201],
 [1],
 [0.5866336633663366],
 [0],
 [0]]

In [67]:
testAccuracy.consumersAccuracy().getInfo()

[[0,
  0.003115264797507788,
  0.13142857142857142,
  0.09070796460176991,
  0.7839195979899497,
  0.08695652173913043,
  0.002105263157894737,
  0.9022556390977443,
  0.1,
  1,
  0.38333333333333336,
  0.6037735849056604,
  0.3448275862068966,
  0.2857142857142857,
  0.7915309446254072,
  0.024390243902439025,
  0.5895522388059702,
  0,
  0]]

SVM

In [68]:
train_accuracy1 = trained.confusionMatrix()

In [69]:
train_accuracy1.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 633, 11, 0, 0, 0, 58, 0, 0, 0, 0, 0, 97, 0, 18, 0, 69, 0, 4],
 [0, 6, 608, 0, 0, 0, 1, 0, 0, 4, 0, 0, 1, 1, 0, 0, 37, 0, 461],
 [0, 0, 0, 4297, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0],
 [0, 0, 0, 64, 2160, 1, 0, 56, 0, 0, 2, 0, 0, 9, 0, 9, 0, 1, 0],
 [0, 0, 0, 0, 0, 52, 5, 0, 0, 0, 4, 15, 60, 0, 2, 0, 1, 0, 0],
 [0, 36, 2, 0, 0, 0, 651, 0, 0, 0, 0, 3, 507, 0, 26, 0, 105, 1, 0],
 [0, 0, 0, 0, 77, 0, 0, 804, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 2, 0, 0, 0, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 0, 0, 116, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 13, 3, 0, 1, 0, 0, 297, 0, 0, 0, 0, 0, 0, 0, 6],
 [0, 0, 7, 0, 0, 11, 3, 0, 0, 5, 5, 145, 38, 1, 2, 0, 1, 0, 14],
 [0, 11, 3, 0, 0, 0, 136, 7, 0, 0, 0, 0, 4282, 0, 31, 0, 3, 0, 0],
 [0, 0, 0, 0, 4, 6, 0, 0, 0, 3, 4, 2, 10, 259, 0, 0, 0, 0, 78],
 [0, 62, 0, 0, 0, 0, 109, 0, 0, 0, 0, 0, 110, 0, 710, 0, 116, 1, 0],
 [0, 0, 0, 73, 12, 1, 0, 0, 0, 0, 

In [70]:
train_accuracy1.accuracy().getInfo()

0.8477019978348588

In [71]:
train_accuracy1.kappa().getInfo()

0.8239095239853439

In [72]:
train_accuracy1.producersAccuracy().getInfo()

[[0],
 [0.7112359550561798],
 [0.5433422698838248],
 [0.9949062282935864],
 [0.9383145091225021],
 [0.37410071942446044],
 [0.48910593538692715],
 [0.9125993189557321],
 [0.9459459459459459],
 [0.9666666666666667],
 [0.928125],
 [0.625],
 [0.9572993516655488],
 [0.7076502732240437],
 [0.6407942238267148],
 [0.7142857142857143],
 [0.7288842544316997],
 [0.6759259259259259],
 [0.9253218884120171]]

In [73]:
train_accuracy1.consumersAccuracy().getInfo()

[[0,
  0.7922403003754693,
  0.8456189151599444,
  0.9673570463755066,
  0.9452954048140044,
  0.7027027027027027,
  0.6072761194029851,
  0.9262672811059908,
  1,
  0.90625,
  0.9488817891373802,
  0.8734939759036144,
  0.8157744332253762,
  0.9487179487179487,
  0.885286783042394,
  0.9361702127659575,
  0.6753623188405797,
  0.9798657718120806,
  0.6561168594035302]]

In [74]:
validacion1 = datasetval.classify(trained)

In [75]:
testAccuracy1 = validacion1.errorMatrix('ID', 'classification')

In [76]:
testAccuracy1.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 19, 0, 0, 1, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 9, 0, 21],
 [0, 0, 0, 47, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 7, 0],
 [0, 4, 7, 462, 233, 0, 10, 10, 19, 1, 0, 0, 5, 10, 8, 109, 0, 7, 0],
 [0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 4, 16, 51, 116, 19, 7, 169, 0, 15, 63, 9, 21, 21, 10, 0, 2, 9, 53],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 6, 32, 0, 36, 12, 3, 7, 0, 17, 5, 15, 41, 13, 6, 0, 0, 0, 14],
 [0, 10, 35, 9, 20, 10, 14, 11, 0, 11, 10, 30, 137, 9, 31, 0, 10, 0, 30],
 [0, 0, 8, 0, 0, 0, 13, 0, 0, 2, 0, 0, 17, 0, 0, 0, 10, 0, 0],
 [0, 9, 36, 14, 3, 18, 0, 16, 0, 56, 47, 0, 21, 55, 0, 0, 10, 0, 138],
 [0, 319, 18, 7, 20, 0, 461, 2, 0, 9, 0, 10, 266, 0, 222, 6, 130, 40, 48],
 [0, 0, 0, 0, 0, 0, 0

In [77]:
testAccuracy1.accuracy().getInfo()

0.1837999572558239

In [78]:
testAccuracy1.kappa().getInfo()

0.13233280008204165

In [79]:
testAccuracy1.producersAccuracy().getInfo()

[[0],
 [0],
 [0.3114754098360656],
 [0.7121212121212122],
 [0.2632768361581921],
 [0.6666666666666666],
 [0],
 [0.28888888888888886],
 [0],
 [0.6666666666666666],
 [0.024154589371980676],
 [0.07957559681697612],
 [0.34],
 [0.13002364066193853],
 [0.14249037227214378],
 [1],
 [0.14356435643564355],
 [0],
 [0]]

In [80]:
testAccuracy1.consumersAccuracy().getInfo()

[[0,
  0,
  0.10326086956521739,
  0.07592891760904685,
  0.5143487858719646,
  0.029411764705882353,
  0,
  0.786046511627907,
  0,
  0.014814814814814815,
  0.04,
  0.3614457831325301,
  0.0264797507788162,
  0.5,
  0.7326732673267327,
  0.008064516129032258,
  0.24369747899159663,
  0,
  0]]

SCART

In [81]:
train_accuracy = smc.confusionMatrix()

In [82]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 890, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 4319, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 2302, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 139, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1331, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 881, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 74, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 320, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 232, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4473, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 366, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1108, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 308, 0, 0, 0],
 [0, 

In [83]:
train_accuracy.accuracy().getInfo()

1

In [84]:
train_accuracy.kappa().getInfo()

1

In [85]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [86]:
train_accuracy.consumersAccuracy().getInfo()

[[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [87]:
validated = datasetval.classify(smc)

In [88]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 0.1297835111618042,
  'B10': 278.9761047363281,
  'B11': 276.3595275878906,
  'B2': 0.10309768468141556,
  'B3': 0.06259501725435257,
  'B4': 0.046153344213962555,
  'B5': 0.0707976296544075,
  'B6': 0.03525298461318016,
  'B7': 0.027852408587932587,
  'B8': 0.05712661147117615,
  'B9': 0.0011301300255581737,
  'BQA': 2976,
  'ID': 3,
  'NDVI': 0.21072322130203247,
  'classification': 3,
  'date': 20190321,
  'doy': 80,
  'month': 3}}

In [89]:
test_accuracy = validated.errorMatrix('ID', 'classification')

In [90]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 21],
 [0, 0, 0, 28, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 5, 1, 4],
 [0, 0, 0, 348, 434, 6, 0, 3, 21, 0, 0, 0, 0, 7, 0, 66, 0, 0, 0],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 51, 4, 104, 0, 0, 241, 0, 0, 38, 3, 0, 60, 0, 17, 0, 0, 67],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 33, 35, 0, 9, 2, 18, 10, 0, 0, 28, 15, 16, 21, 1, 0, 0, 4, 15],
 [0, 28, 12, 0, 2, 7, 133, 0, 0, 6, 7, 58, 14, 8, 43, 0, 19, 14, 26],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0, 2, 0, 8],
 [0, 17, 167, 0, 4, 10, 6, 4, 0, 10, 3, 5, 3, 55, 0, 0, 2, 0, 137],
 [0, 539, 17, 9, 1, 1, 231, 0, 0, 0, 0, 14, 47, 0, 473, 4, 171, 51, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [91]:
test_accuracy.accuracy().getInfo()

0.3400299209232742

In [92]:
test_accuracy.kappa().getInfo()

0.2774528257210206

In [93]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.5],
 [0.639344262295082],
 [0.42424242424242425],
 [0.4903954802259887],
 [1],
 [1],
 [0.41196581196581195],
 [0.08333333333333333],
 [0.6666666666666666],
 [0.13526570048309178],
 [0.15384615384615385],
 [0.8],
 [0.13002364066193853],
 [0.30359435173299104],
 [1],
 [0.4603960396039604],
 [0],
 [0]]

In [94]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.0014992503748125937,
  0.10893854748603352,
  0.06572769953051644,
  0.7736185383244206,
  0.10344827586206896,
  0.002325581395348837,
  0.9341085271317829,
  0.045454545454545456,
  0.1111111111111111,
  0.3684210526315789,
  0.48333333333333334,
  0.3076923076923077,
  0.36423841059602646,
  0.8211805555555556,
  0.008849557522123894,
  0.48186528497409326,
  0,
  0]]